# Study the effect of the GENIE knobs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import localSettings as ls
from load_data_run123 import *
from unblinding_far_sideband import *
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy.optimize import curve_fit
import seaborn as sns
from ccncpi0_analysis import *
from sys_functions import *

In [ ]:
knobList = np.array(["AGKYpT1pi_Genie","AGKYxF1pi_Genie","AhtBY_Genie","AxFFCCQEshape_Genie","BhtBY_Genie",
            "CV1uBY_Genie","CV2uBY_Genie","DecayAngMEC_Genie","EtaNCEL_Genie","FrAbs_N_Genie",
            "FrAbs_pi_Genie","FrCEx_N_Genie","FrCEx_pi_Genie","FrInel_N_Genie","FrInel_pi_Genie",
            "FrPiProd_N_Genie","FrPiProd_pi_Genie","FracDelta_CCMEC_Genie","FracPN_CCMEC_Genie","MFP_N_Genie",
            "MFP_pi_Genie","MaCCQE_Genie","MaCCRES_Genie","MaNCEL_Genie","MaNCRES_Genie",
            "MvCCRES_Genie","MvNCRES_Genie","NonRESBGvbarnCC1pi_Genie","NonRESBGvbarnCC2pi_Genie","NonRESBGvbarnNC1pi_Genie",
            "NonRESBGvbarnNC2pi_Genie","NonRESBGvbarpCC1pi_Genie","NonRESBGvbarpCC2pi_Genie","NonRESBGvbarpNC1pi_Genie","NonRESBGvbarpNC2pi_Genie",
            "NonRESBGvnCC1pi_Genie","NonRESBGvnCC2pi_Genie","NonRESBGvnNC1pi_Genie","NonRESBGvnNC2pi_Genie","NonRESBGvpCC1pi_Genie",
            "NonRESBGvpCC2pi_Genie","NonRESBGvpNC1pi_Genie","NonRESBGvpNC2pi_Genie","NormCCMEC_Genie","NormNCMEC_Genie",
            "RDecBR1eta_Genie","RDecBR1gamma_Genie","RPA_CCQE_Genie","Theta_Delta2Npi_Genie","TunedCentralValue_Genie",
            "VecFFCCQEshape_Genie","XSecShape_CCMEC_Genie","splines_general_Spline"])

In [ ]:
ntuple_path = ls.ntuple_path+"genie_knobs/"

In [ ]:
my_ccncpi0_analysis=ccncpi0_analysis(ntuple_path, 
                 fold="nuselection", 
                 tree="NeutrinoSelectionFilter", 
                 fcc1="prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2", 
                 fcc3=None, 
                 fnc1="prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2", 
                 fnc3=None)

In [ ]:
Eff_ccnc(my_ccncpi0_analysis.pi0s, 
          weights=my_ccncpi0_analysis.pi0s["weightSplineTimesTune"], 
          var='pi0_e',
          num_query=my_ccncpi0_analysis.CCNCSEL, 
          den_query=my_ccncpi0_analysis.ACCEPTANCE, 
          bin_edges=[0.1, 1.5], #only one bin, overall efficiency
          num_dem=True)

In [ ]:
# in order to get up and down variations just call sampleSystematics twice, with the two different vectors
cv, up_variations = sampleSystematics(my_ccncpi0_analysis.pi0s, 
                  Eff_ccnc, 
                  var_weight_sys="weightsGenieUp", 
                  var_weight_cv='weightSplineTimesTune', 
                  var='pi0_e',
                  num_query=my_ccncpi0_analysis.CCNCSEL, 
                  den_query=my_ccncpi0_analysis.ACCEPTANCE, 
                  bin_edges=[0.1, 1.5],
                  num_dem=False)

cv, dn_variations = sampleSystematics(my_ccncpi0_analysis.pi0s, 
                  Eff_ccnc, 
                  var_weight_sys="weightsGenieDn", 
                  var_weight_cv='weightSplineTimesTune', 
                  var='pi0_e',
                  num_query=my_ccncpi0_analysis.CCNCSEL, 
                  den_query=my_ccncpi0_analysis.ACCEPTANCE, 
                  bin_edges=[0.1, 1.5],
                  num_dem=False)

rel_up_variations = up_variations/cv
rel_dn_variations = dn_variations/cv

In [ ]:
plt.plot(rel_up_variations[:, 0], label='up variations')
plt.plot(rel_dn_variations[:, 0], label='down variations')
plt.hlines(1, 0, len(rel_up_variations[:, 2]), 
           colors='brown', 
           linestyles='dashed')
plt.legend(frameon=False)
plt.xlabel('Genie Knob index')
plt.ylabel('Ratio wrt to central value')
plt.title('CC $\pi^0$ selection efficiency')

In [ ]:
plt.plot(rel_up_variations[:, 1], label='up variations')
plt.plot(rel_dn_variations[:, 1], label='down variations')
plt.hlines(1, 0, len(rel_up_variations[:, 2]), 
           colors='brown', 
           linestyles='dashed')
plt.legend(frameon=False)
plt.xlabel('Genie Knob index')
plt.ylabel('Ratio wrt to central value')
plt.title('NC $\pi^0$ selection efficiency')

In [ ]:
plt.plot(rel_up_variations[:, 2], label='up variations')
plt.plot(rel_dn_variations[:, 2], label='down variations')
plt.hlines(1, 0, len(rel_up_variations[:, 2]), 
           colors='brown', 
           linestyles='dashed')
plt.legend(frameon=False)
plt.xlabel('Genie Knob index')
plt.ylabel('Ratio wrt to central value')
plt.title('NC/CC $\pi^0$ selection efficiency ratio')

In [ ]:
#find indices with large variations
knobList[(np.abs(rel_dn_variations[:, 0]) > 1.01)]